# Boston Crime Postgres Workflow

   In this project, the workflow will demonstrate the creation of a database, users, groups, and data transfer of a CSV to a PostgreSQL server. Using the psycopg2 module; storing and querying data in a postgres server for analysis in Python makes for a very powerful tool.
   
   Lets start by importing psycopg2, then after connecting to the server and creating the schema, the database can be made.

In [ ]:
import psycopg2
conn = psycopg2.connect(dbname='myles',user='postgres')
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="crime_db",user="postgres")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")
conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect(dbname="crime_db",user="postgres")
cur = conn.cursor()

## Inspecting the Data

Now that the database and schema are up and running, lets take a look at the data before sending it to the db.

In [ ]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

In [ ]:
print(col_headers)
print("\n")
print(first_row)

Before creating a table, I want to explore where I can optimize data types as this is running on my local machine. Specifically, I want to look at columns that have a low unique value count. With columns like 'description', 'day_of_the_week', and 'date'; Enumerating or changing to an appropriate type that will not allocate memory for unused space ( ex. int32 vs int 64 )

In [4]:
def get_col_set(csv_file, col_index):
    unique = set()
    with open(csv_file) as file:
        reader = csv.reader(file)
        next(reader)
        for i in reader:
            unique.add(i[col_index])
    return unique

In [5]:
for i in range(7):
    x = get_col_set("boston.csv",i)
    print(col_headers[i]+": " + str(len(x)), sep="\t")

NameError: name 'csv' is not defined

Looking at the results above, enumerating 'day_of_the_week' seems to be the best option as there are seven unique values which can be translated into integers. To retain more direct information regarding the temporal data, the 'date' column will be converted to the date object. The most ambiguous column here is 'description'. 

In [ ]:
col_values = []
desc = get_col_set("boston.csv",2)
for i in desc:
    col_values.append(i)

In [ ]:
long = max(col_values,key=len)
short = min(col_values,key=len)
print(long)
print(len(long))
print("\n")
print(short)
print(len(short))

Unfortunately, the description column seems to range from one word, to an entire phrase with symbols like '-'. For this reason the column will remain a text type. 

## Creating Types & Moving to the Database

In [ ]:
cur.execute("""CREATE TYPE day_of_week AS ENUM ( \
'Sunday', 
'Monday',
'Tuesday',
'Wednesday',
'Thursday',
'Friday',
'Saturday');""")

In [ ]:
cur.execute("""CREATE TABLE crimes.boston_crimes ( \
                    incident_number INTEGER PRIMARY KEY,
                    offense_code BIGINT,
                    description text,
                    date DATE,
                    day day_of_week,
                    lat FLOAT,
                    long Float);""")
conn.commit()

In [ ]:
with open('boston.csv') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;",file)
    
conn.commit()    

## Users, Groups, and Permissions

The users and groups schema consists of two groups; readonly, and readwrite. I've created and assigned two users to these groups; data_analyst and data_scientist, not only to demonstrate their use, but to have a user/group that only allows me to read the data to prevent any INSERT or DELETE mistakes

In [ ]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")
conn.commit()

In [ ]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
conn.commit()

In [ ]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst; ")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist; ")
conn.commit()

In [ ]:
cur.execute("""SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite';""")
cur.fetchall()

## Querying the data 

In [ ]:
conn = psycopg2.connect(dbname="crime_db",user='data_analyst',password="secret1")
cur = conn.cursor()

Proportion of crime for each day of the week.

In [ ]:
cur.execute("""
               SELECT Day, ROUND(CAST(COUNT(offense_code) AS numeric)/
               CAST((SELECT COUNT(*) FROM crimes.boston_crimes) AS numeric), 2) week_avg
               FROM crimes.boston_crimes
               GROUP by Day
               ORDER BY week_avg DESC;
                """)
cur.fetchall()

Description for top 10 offense codes.

In [ ]:
cur.execute("""WITH top_10 AS (
               
               SELECT offense_code, COUNT(*) total
               FROM crimes.boston_crimes
               GROUP BY offense_code
               ORDER BY total DESC
               LIMIT 10)
               
               SELECT b.description,t.total FROM crimes.boston_crimes b,top_10 t
               WHERE t.offense_code = b.offense_code
               LIMIT 10;""")

cur.fetchall()

In [ ]:
conn.close()